In [2]:
# import sagemaker
import pandas as pd
from io import StringIO
import boto3

In [9]:
session = sagemaker.session.Session()
bucket = session.default_bucket()
client = boto3.client('s3')
response = client.get_object(Bucket=bucket, Key='data/data.csv')
body=response['Body']

data = body.read().decode('utf-8')
df = pd.read_csv(StringIO(data))
print(df.head())

   deceased  subject_id  gender   race arrival_transport  existing_doses  \
0         0    14188788       1  OTHER         AMBULANCE               0   
1         0    19659841       1  OTHER         AMBULANCE               1   
2         0    11074777       1  WHITE           WALK IN               4   
3         0    11075647       0  WHITE           WALK IN               1   
4         0    11858930       1  WHITE        HELICOPTER               0   

   medicine_dispensations  temperature  heartrate  resprate  o2sat  sbp  dbp  \
0                       6          NaN        NaN       NaN    NaN  NaN  NaN   
1                      12          NaN        NaN       NaN    NaN  NaN  NaN   
2                       0          NaN        NaN       NaN    NaN  NaN  NaN   
3                       0          NaN        NaN       NaN    NaN  NaN  NaN   
4                       6          NaN        NaN       NaN    NaN  NaN  NaN   

  pain  acuity  
0  NaN     NaN  
1  NaN     NaN  
2  NaN     

/tmp/ipykernel_10837/2122273395.py:15: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data))


In [9]:
df = pd.read_csv('data.csv')

/tmp/ipykernel_21801/1507164070.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205504 entries, 0 to 205503
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   deceased                205504 non-null  int64  
 1   subject_id              205504 non-null  int64  
 2   gender                  205504 non-null  int64  
 3   race                    205504 non-null  object 
 4   arrival_transport       205504 non-null  object 
 5   existing_doses          205504 non-null  int64  
 6   medicine_dispensations  205504 non-null  int64  
 7   temperature             191025 non-null  float64
 8   heartrate               194174 non-null  float64
 9   resprate                192573 non-null  float64
 10  o2sat                   192534 non-null  float64
 11  sbp                     193632 non-null  float64
 12  dbp                     193241 non-null  float64
 13  pain                    197462 non-null  object 
 14  acuity              

In [11]:
for column in df.columns:
    print(column)
    missing_values = df[column].isna().sum()
    print(f'Missing values: {missing_values}')
    print(f'Percentage missing values: {missing_values / len(df[column]) : .2%}', '\n')

deceased
Missing values: 0
Percentage missing values:  0.00% 

subject_id
Missing values: 0
Percentage missing values:  0.00% 

gender
Missing values: 0
Percentage missing values:  0.00% 

race
Missing values: 0
Percentage missing values:  0.00% 

arrival_transport
Missing values: 0
Percentage missing values:  0.00% 

existing_doses
Missing values: 0
Percentage missing values:  0.00% 

medicine_dispensations
Missing values: 0
Percentage missing values:  0.00% 

temperature
Missing values: 14479
Percentage missing values:  7.05% 

heartrate
Missing values: 11330
Percentage missing values:  5.51% 

resprate
Missing values: 12931
Percentage missing values:  6.29% 

o2sat
Missing values: 12970
Percentage missing values:  6.31% 

sbp
Missing values: 11872
Percentage missing values:  5.78% 

dbp
Missing values: 12263
Percentage missing values:  5.97% 

pain
Missing values: 8042
Percentage missing values:  3.91% 

acuity
Missing values: 5076
Percentage missing values:  2.47% 



In [12]:
df = df.dropna()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185294 entries, 8025 to 205503
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   deceased                185294 non-null  int64  
 1   subject_id              185294 non-null  int64  
 2   gender                  185294 non-null  int64  
 3   race                    185294 non-null  object 
 4   arrival_transport       185294 non-null  object 
 5   existing_doses          185294 non-null  int64  
 6   medicine_dispensations  185294 non-null  int64  
 7   temperature             185294 non-null  float64
 8   heartrate               185294 non-null  float64
 9   resprate                185294 non-null  float64
 10  o2sat                   185294 non-null  float64
 11  sbp                     185294 non-null  float64
 12  dbp                     185294 non-null  float64
 13  pain                    185294 non-null  object 
 14  acuity           

deceased - cleaned before loading
subject_id - drop, not relevant to the prediction model
race - encode multi class (binary encoder)
arrival transport - encode multi class (one hot encoder)
existing doses - rescale (min max scaler)
medicine dispensations - rescale(min max scaler)
temperature - good
heartrate - good
resprate - good
o2sat - good
sbp - drop values higher than 300 (even then, these are extreme)
dbp - drop values higher than 300 (even then, these are extreme)
pain - drop because there are a lot of unusable values
acuity - good

In [15]:
for column in df.columns:
    print(df[column].value_counts())

0    170049
1     15245
Name: deceased, dtype: int64
19903802    1
19976339    1
10590663    1
13041508    1
14044558    1
           ..
10554304    1
11062044    1
14462434    1
14714653    1
13716295    1
Name: subject_id, Length: 185294, dtype: int64
0    100296
1     84998
Name: gender, dtype: int64
WHITE                                        108247
BLACK/AFRICAN AMERICAN                        23823
OTHER                                         10808
ASIAN                                          4601
HISPANIC/LATINO - PUERTO RICAN                 4293
WHITE - OTHER EUROPEAN                         4190
ASIAN - CHINESE                                3880
UNKNOWN                                        3142
HISPANIC/LATINO - DOMINICAN                    3008
BLACK/CAPE VERDEAN                             2610
BLACK/AFRICAN                                  2250
WHITE - RUSSIAN                                2006
BLACK/CARIBBEAN ISLAND                         1535
HISPANIC OR LATINO 

In [16]:
print(df['pain'].unique())

[')' '+' '-' '.' '0' 0 1 2 3 4 5 6 '6' '7' '8' '9' '?' 'U' 'c' 'o' 's' 'u'
 ' 0' ' 4' ' 6' '++' '-0' '.0' '.3' '.5' '0 ' '0-' '0.' '0/' '00' '01'
 '03' '06' '07' '08' '09' '10' '11' '12' '13' '14' '15' '16' '18' '19'
 '2,' '2.' '20' '23' '3 ' '5 ' '51' '54' '56' '6 ' '61' '68' '6`' '7/'
 '73' '75' '8 ' '8,' '8/' '88' '9 ' '9+' '90' '95' '97' '98' '<1' '??'
 'RA' 'UA' 'na' 'no' 'o3' 'ok' 'ua' 'un' 'ut' ' 10' '0  ' '0, ' '0-1'
 '0-2' '0-5' '0-7' '0-8' '0-9' '0..' '0.5' '1 0' '1-2' '1-3' '1-7' '1.2'
 '1.5' '1/2' '10 ' '10+' '10.' '10/' '100' '134' '1`0' '2-3' '2-4' '2-6'
 '2-7' '2-8' '2-9' '2. ' '2.5' '3  ' '3-4' '3-5' '3-6' '3-7' '3.5' '4-5'
 '4-8' '4. ' '4.5' '4/9' '5-6' '5-8' '5-9' '5.5' '5.8' '5/7' '5/8' '5/9'
 '6-7' '6-8' '6-9' '6.5' '7-6' '7-8' '7-9' '7.5' '7/8' '8  ' '8-9' '8.5'
 '8.6' '8.7' '8/6' '8/9' '9  ' '9-8' '9.4' '9.5' '9.7' '9.9' '9/6' '>10'
 'AMS' 'Bad' 'NAD' 'Pt ' 'U.A' 'U/A' 'UTA' '___' '`10' 'bad' 'low' 'mod'
 'pta' 'u/a' 'unk' 'uta' 'uto' 'uts' 'yes' '"11"' '"12"' '"1

In [17]:
df = df.drop(axis=1, columns=['subject_id', 'pain'])

In [18]:
dummies = pd.get_dummies(df['arrival_transport'], prefix='arrival_transport', drop_first=True)
df = df.join(dummies)
df = df.drop('arrival_transport', axis=1)

In [21]:
from category_encoders import BinaryEncoder
df = BinaryEncoder(cols=['race'], drop_invariant=True).fit_transform(df)
df.head()
# df = df.drop('race', axis=1)

,deceased,gender,race_0,race_1,race_2,race_3,race_4,race_5,existing_doses,medicine_dispensations,...,heartrate,resprate,o2sat,sbp,dbp,acuity,arrival_transport_HELICOPTER,arrival_transport_OTHER,arrival_transport_UNKNOWN,arrival_transport_WALK IN
8025,0,1,0,0,0,0,0,1,0,0,...,92.0,16.0,98.0,100.0,66.0,3.0,0,0,0,0
8026,0,1,0,0,0,0,0,1,13,1,...,50.0,18.0,97.0,182.0,82.0,1.0,0,0,0,0
8028,0,1,0,0,0,0,1,0,0,7,...,71.0,16.0,97.0,143.0,83.0,2.0,0,0,0,0
8029,0,1,0,0,0,0,0,1,0,5,...,65.0,16.0,97.0,114.0,67.0,1.0,0,0,0,0
8030,0,0,0,0,0,0,1,0,3,1,...,76.0,16.0,100.0,130.0,76.0,3.0,0,0,0,0


In [22]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[ ['existing_doses', 'medicine_dispensations'] ] = scaler.fit_transform(df[ ['existing_doses', 'medicine_dispensations'] ])


In [23]:
# sbp - drop values higher than 300 (even then, these are extreme)
# dbp - drop values higher than 300 (even then, these are extreme)
df = df[ (df['sbp'] <= 300) & (df['dbp'] <= 300) ]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185111 entries, 8025 to 205503
Data columns (total 21 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   deceased                      185111 non-null  int64  
 1   gender                        185111 non-null  int64  
 2   race_0                        185111 non-null  int64  
 3   race_1                        185111 non-null  int64  
 4   race_2                        185111 non-null  int64  
 5   race_3                        185111 non-null  int64  
 6   race_4                        185111 non-null  int64  
 7   race_5                        185111 non-null  int64  
 8   existing_doses                185111 non-null  float64
 9   medicine_dispensations        185111 non-null  float64
 10  temperature                   185111 non-null  float64
 11  heartrate                     185111 non-null  float64
 12  resprate                      185111 non-

In [30]:
print(sum(df.deceased)/len(df.deceased))

0.08228036151282203


In [27]:
len(df.deceased)

185111

In [29]:
sum(df['deceased'])

15231

In [35]:
X = df.drop(columns='deceased')
y = df['deceased']

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y)

In [37]:
data_train, data_test = train_test_split(df, test_size=.2, stratify=df['deceased'])

In [38]:
print(sum(data_train.deceased)/len(data_train.deceased))

0.08228215655556156


In [39]:
print(sum(data_test.deceased)/len(data_test.deceased))

0.08227318153580207


In [40]:
print(sum(y_train)/len(y_train))

0.08228215655556156


In [41]:
print(sum(y_test)/len(y_test))

0.08227318153580207


Elegi el pytorch tabular como libreria y modelo de prediccion (clasificacion con % de seguridad), porque de acuerdo a las notaas de la clase, los algoritmos de deep learning mejoran con mayores cantidades de datos, y para este prooyecto, esto puede ser unaa mejora a futuro. Tmbien va a ser interesaante revisar el desempe;o de este modelo frente al Boosting, que es aahora el estandar que elegi para comparar el proyecto. <br><br> Aqui un ejemplo de uso de la libreria https://analyticsindiamag.com/how-to-handle-tabular-data-for-deep-learning-using-pytorch-tabular/

In [ ]:
!pip install pytorch_tabular